In [77]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from load_clean_data import load_clean_data

In [78]:
# Create a SparkSession
spark = SparkSession.builder.appName("Task4").getOrCreate()


In [79]:
# Load the data
df = load_clean_data("./data", spark)


In [80]:
# Group by channel and calculate the mean and median of the trusting points
df_grouped = df.groupBy("_channel").agg(
    F.mean("_trust").alias("mean_trusting_points"),
    F.median("_trust").alias("median_trusting_points")
)


In [81]:
# Show the results
df_grouped.show(df_grouped.count(), False)


+----------------+--------------------+----------------------+
|_channel        |mean_trusting_points|median_trusting_points|
+----------------+--------------------+----------------------+
|prizeplank      |1.0                 |1.0                   |
|instagc         |0.9086886074429764  |0.9259                |
|bitcoinget      |0.9577211864406794  |0.9615                |
|tremorgames     |0.8287998293970339  |0.8108                |
|coinworker      |0.8390619450317137  |0.8571                |
|keeprewarding   |0.866699999999999   |0.8667                |
|inboxpounds     |0.9375              |0.9375                |
|neodev          |0.8399505561040375  |0.8378                |
|zoombucks       |0.9283105102818009  |0.9189                |
|cotter          |0.9188999999999999  |0.9189                |
|surveymad       |0.8474142857142861  |0.8182                |
|tasks4dollars   |0.8106617647058824  |0.8125                |
|globalactioncash|0.7999999999999998  |0.8             

In [82]:
#stop the session
spark.stop()